In [1]:
from ephys_queries import (
    select_spike_times
)
from ephys_queries import db_setup_core
from dotenv import load_dotenv

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
from spiketimes.df.population import population_coupling_df

In [4]:
data_dir = Path(".").absolute().parent / "data"
dfb = pd.read_csv(data_dir / "baseline.csv")

In [5]:
load_dotenv()
engine, metadata = db_setup_core()
group_names = ["acute_citalopram", "acute_saline", "shock", 
               "sham", "acute_cit", "acute_sal"]
block_name = "pre"
fs = 30000

df_spiketimes = (
    select_spike_times(
        engine, metadata, 
        block_name=block_name, 
        group_names=group_names,
    )
    .assign(
        spiketimes= lambda x: x["spike_time_samples"].divide(fs)
    )
    .drop("spike_time_samples", axis=1)
    .merge(dfb[["cluster", "neuron_id", "session_name", "group_name"]])
    .loc[lambda x: x["cluster"] != "no_baseline"]
)

In [6]:
session_names = df_spiketimes["session_name"].unique()

In [7]:
big_frames = []
BIN = 0.001
NUM_LAGS=400

frames = []
for session in session_names:
    frames.append(
        df_spiketimes
        .loc[lambda x: x["session_name"]==session]
        .pipe(
            lambda x: population_coupling_df(
                x,
                binsize=BIN,
                num_lags=NUM_LAGS,
                spiketimes_col="spiketimes", 
                spiketrain_col="neuron_id",
                return_all=False
                )
            )
            .assign(session_name=session,
                   num_lags=NUM_LAGS,
                   )
        )
df = pd.concat(frames).assign(binsize=BIN)

In [15]:
(
    df
    .merge(dfb)
    .to_csv(data_dir / "population_coupling.csv", index=False)
)